### Passo 1 - Importar Arquivos e Bibliotecas

In [2]:
# Imports
import pandas as pd
from pathlib import Path
import win32com.client as win32
import time

### Passo 2 - Definir Criar uma Tabela para cada Loja e Definir o dia do Indicador

In [3]:
# Ler as bases de dados
emails = pd.read_excel(r'Bases de Dados\Emails.xlsx')
vendas = pd.read_excel(r'Bases de Dados\Vendas.xlsx')
lojas = pd.read_csv(r'Bases de Dados\Lojas.csv', encoding='latin1', sep=';')

In [4]:
# Mesclar as duas tabelas
vendas = vendas.merge(lojas, on="ID Loja")

# Pega o dia da operação
dia = vendas['Data'].max()
# print(f'{dia.day}/{dia.month}/{dia.year}')

In [5]:
meta_faturamento_dia = 1000
meta_faturamento_ano = 1650000
meta_qtdprodutos_dia = 4
meta_qtdprodutos_ano = 120
meta_ticket_dia = 500
meta_ticket_ano = 500

In [6]:
# Criando e preenchendo um dicionário de dataframes com as keys sendo o nome de cada loja
dic_lojas = {}
for loja in lojas["Loja"]:
    dic_lojas[loja] = vendas.loc[vendas["Loja"] == loja , :]

### Passo 3 - Salvar a planilha na pasta de backup

In [7]:
caminho = Path(r'Backup Arquivos Lojas')
arquivos = caminho.iterdir()

# Criar uma lista com o nome das lojas
lista_arquivos = [arquivo.name for arquivo in arquivos]


# Verificar a existência das pastas
for loja in dic_lojas:
    if loja not in lista_arquivos:
        criar = caminho / loja
        criar.mkdir()
    nome_arquivo = f'{dia.day}-{dia.month}-{dia.year}-{loja}.xlsx'
    local = caminho / loja / nome_arquivo
    dic_lojas[loja].to_excel(local)

### Passo 4 - Calcular o indicador para 1 loja

In [8]:
for loja in dic_lojas:
    vendas_loja = dic_lojas[loja]
    vendas_dia = vendas_loja.loc[vendas_loja['Data'] == dia, :]
    
    # Calculando o faturamento
    faturamento_ano = vendas_loja['Valor Final'].sum()
    faturamento_dia = vendas_dia['Valor Final'].sum()

    # Calculando as quantidades de produtos diferentes
    qtd_prod_ano = len(vendas_loja['Produto'].unique())
    qtd_prod_dia = len(vendas_dia['Produto'].unique())

    # Calculando o ticket médio
    valor_venda_ano = vendas_loja.groupby('Código Venda').sum()
    ticket_medio_ano = valor_venda_ano['Valor Final'].mean()
    valor_venda_dia = vendas_dia.groupby('Código Venda').sum()
    ticket_medio_dia = valor_venda_dia['Valor Final'].mean()

    # Pegando novamente o nome dos arquivos
    nome_arquivo = f'{dia.day}-{dia.month}-{dia.year}-{loja}.xlsx'

    # Montando o e-mail
    outlook = win32.Dispatch('outlook.application')

    nome_email = emails.loc[emails["Loja"] == loja, "Gerente"].values[0]
    mail = outlook.CreateItem(0)
    mail.To = emails.loc[emails["Loja"] == loja, "E-mail"].values[0]
    mail.Subject = 'One Page - {}/{}/{} - Loja: {}'.format(dia.day, dia.month, dia.year, loja)

    mail.HTMLBody = f"""
    <p>Bom dia, <b>{nome_email}</b></p>

    <p> O <b>resultado</b> de ontem <b>({dia.day}/{dia.month}/{dia.year})</b> da loja <b>{loja}</b> foi:<p>

    <table style="background: black; color: blue;">
        <tr>
            <th>Indicador</th>
            <th>Valor Dia</th>
            <th>Meta Dia</th>
            <th>Cenário Dia</th>
        </tr>
        <tr>
            <td>Faturamento</td>
            <td>{faturamento_dia}</td>
            <td>{meta_faturamento_dia}</td>
            <td style="color: {"green" if faturamento_dia >= meta_faturamento_dia else "red"};">◙</td>
        </tr>
        <tr>
            <td>Diversidade de Produtos</td>
            <td>{qtd_prod_dia}</td>
            <td>{meta_qtdprodutos_dia}</td>
            <td style="color: {"green" if qtd_prod_dia >= meta_qtdprodutos_dia else "red"};">◙</td>
        </tr>
        <tr>
            <td>Ticket Médio</td>
            <td>{ticket_medio_dia}</td>
            <td>{meta_ticket_dia}</td>
            <td style="color: {"green" if ticket_medio_dia >= meta_ticket_dia else "red"};">◙</td>
        </tr>
    </table>
    <br>
    <table style="background: black; color: blue;">
        <tr>
            <th>Indicador</th>
            <th>Valor Ano</th>
            <th>Meta Ano</th>
            <th>Cenário Ano</th>
        </tr>
        <tr>
            <td>Faturamento</td>
            <td>{faturamento_ano}</td>
            <td>{meta_faturamento_ano}</td>
            <td style="color: {"green" if faturamento_ano >= meta_faturamento_ano else "red"};">◙</td>
        </tr>
        <tr>
            <td>Diversidade de Produtos</td>
            <td>{qtd_prod_ano}</td>
            <td>{meta_qtdprodutos_ano}</td>
            <td style="color: {"green" if qtd_prod_ano >= meta_qtdprodutos_ano else "red"};">◙</td>
        </tr>
        <tr>
            <td>Ticket Médio</td>
            <td>{ticket_medio_ano}</td>
            <td>{meta_ticket_ano}</td>
            <td style="color: {"green" if ticket_medio_ano >= meta_ticket_ano else "red"};">◙</td>
        </tr>
    </table>
    <p>Segue em <b>anexo</b> a planilha com todos os dados para mais detalhes.</p>

    <p>Qualquer dúvida estou à disposição!</p>
    <p>Att. <b>Erick</b></p>
    """

    attachment =  Path.cwd() / caminho / loja / nome_arquivo
    mail.Attachments.Add(str(attachment))

    mail.Send()

    print(f"Email da loja - {loja} foi enviado com sucesso!")

    time.sleep(60)

Email da loja - Iguatemi Esplanada foi enviado com sucesso!


### Passo 5 - Enviar por e-mail para o gerente

### Passo 6 - Automatizar todas as lojas

### Passo 7 - Criar ranking para diretoria

In [ ]:
# Calculando o ranking do valor total das vendas anuais
faturamento_lojas = vendas.groupby('Loja')[['Loja', 'Valor Final']].sum()
faturamento_lojas = faturamento_lojas.sort_values(by = 'Valor Final', ascending = False)

# Criando os arquivos em excel
nome_arquivo = f'{dia.year}-RankingAnual.xlsx'
faturamento_lojas.to_excel(rf'Backup Arquivos Lojas\{nome_arquivo}')

# Calculando o ranking do valor total das vendas diárias
vendas_dia = vendas.loc[vendas['Data'] == dia, :]
faturamento_lojas_dia = vendas.groupby('Loja')[['Loja', 'Valor Final']].sum()
faturamento_lojas_dia = faturamento_lojas_dia.sort_values(by = 'Valor Final', ascending = False)

# Criando os arquivos em excel
nome_arquivo = f'{dia.day}-{dia.month}-{dia.year}-RankingDiario.xlsx'
faturamento_lojas_dia.to_excel(rf'Backup Arquivos Lojas\{nome_arquivo}')

### Passo 8 - Enviar e-mail para diretoria

In [ ]:
# Montando o e-mail
outlook = win32.Dispatch('outlook.application')

mail = outlook.CreateItem(0)
mail.To = emails.loc[emails["Loja"] == 'Diretoria', 'E-mail'].values[0]
mail.Subject = f'Ranking do dia {dia.day}-{dia.month}-{dia.year}'

mail.Body = f'''Bom dia à todos, 

Melhor loja do dia em faturamento foi a {faturamento_lojas_dia.index[0]} com um faturamento de R${faturamento_lojas_dia.iloc[0,0] : ,.2f}
Pior loja do dia em faturamento foi a {faturamento_lojas_dia.index[-1]} com um faturamento de R${faturamento_lojas_dia.iloc[-1,0] : ,.2f}

Melhor loja do ano em faturamento foi a {faturamento_lojas.index[0]} com um faturamento de R${faturamento_lojas.iloc[0,0] : ,.2f}
Pior loja do ano em faturamento foi a {faturamento_lojas.index[-1]} com um faturamento de R${faturamento_lojas.iloc[-1,0] : ,.2f}

Att.Erick, Full Stack Web Developer
'''
attachment =  Path.cwd() / caminho / f'{dia.day}-{dia.month}-{dia.year}-RankingDiario.xlsx'
attachment1 = Path.cwd() / caminho / f'{dia.year}-RankingAnual.xlsx'
mail.Attachments.Add(str(attachment), str(attachment1))

mail.Send()

print("O email com o ranking da diretoria foi enviado com sucesso!")

NameError: name 'win32' is not defined